In [1]:
from etl.extract import Extract
from etl.transform import Transform
from etl.load import Load
import pandas as pd
import json

In [2]:
extract = Extract()
transform = Transform()
load = Load()

In [3]:
# res = extract.get_api_response("p160991049881")
res = extract.get_api_response("p272193372940")
df_survey_flat = pd.json_normalize(res, record_path="items")
df_survey_flat.columns = df_survey_flat.columns.str.replace(".", "_", regex=False)
df_survey_flat.columns = df_survey_flat.columns.str.replace("$", "_", regex=False)
# df_survey_flat.columns = df_survey_flat.columns.str.replace('"', '_', regex=False)

#converting array values to string in every column
all_columns = list(df_survey_flat) #to get column headers
for i in all_columns:
    val = str(df_survey_flat[i][0])
    if "[" in val:
        df_survey_flat[i] = [','.join(map(str, l)) for l in df_survey_flat[i]]

pd.set_option('display.max_columns', None)
# display(df_survey_flat)

In [4]:
df_survey_flat.head()

,responseid,respid,status,interview_start,interview_end,lastchannel,lastdevicetype,lastrenderingmode,last_touched,first_question_on_last_page_displayed,sectionname,display_name,nmg_title,pastcompleter,Email,info_classification,qi,qv,qvi,response_date,q32countv2,q54,surveyCompTime,footer,surveyname,respondent_study_period_id,master_id,noofemailsent,HasOptedOut,Info_1,Info_6,Info_7,Details_1,Details_2,Details_3,Details_4,Details_5,Details_6,Details_7,Details_8
0,1,1,complete,2022-12-30T04:17:17.227+00:00,2022-12-30T04:23:41.977+00:00,1,1,1,2023-01-06T01:26:39.85+00:00,Internal_Stop,99,Mark Prichard,"CEO, NMG Consulting",2,yaowei2.chun5@nmg-group.com,2,A,E,1333.0,2023-01-02T00:00:00+00:00,2,1,6.0,1,1,89,91,1,3,r,r,yaowei2.chun5@nmg-group.com,r,r,r,r,r,r,11124,124


In [5]:
#### Transform 
df_transform1 = transform.enunciado1(df_survey_flat)

In [6]:
df_transform1.head()

,responseid,respid,status,interview_start,interview_end,lastchannel,lastdevicetype,lastrenderingmode,last_touched,first_question_on_last_page_displayed,sectionname,display_name,nmg_title,pastcompleter,Email,info_classification,qi,qv,qvi,response_date,q32countv2,q54,surveyCompTime,footer,surveyname,respondent_study_period_id,master_id,noofemailsent,HasOptedOut,Info_1,Info_6,Info_7,Details_1,Details_2,Details_3,Details_4,Details_5,Details_6,Details_7,Details_8
0,1,1,complete,2022-12-30T04:17:17.227+00:00,2022-12-30T04:23:41.977+00:00,1,1,1,2023-01-06T01:26:39.85+00:00,Internal_Stop,99,Mark Prichard,"CEO, NMG Consulting",2,yaowei2.chun5@nmg-group.com,2,A,E,1333.0,2023-01-02T00:00:00+00:00,2,1,6.0,1,1,89,91,1,3,r,r,yaowei2.chun5@nmg-group.com,r,r,r,r,r,r,11124,124


In [7]:
from utils import utils as u
import sqlalchemy as db

db = 'test_db'

tbl_name = "SurveyTest"

# u.sql_engine()

u.create_sql_tbl_schema(df_transform1, u.sql_conn(u.sql_engine()), db, tbl_name)

In [8]:
#### Load
load.load_sql(df_transform1, tbl_name)